In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .config('spark.ui.port', '0') \
    .config('spark.shuffle.useOldFetchProtocol', 'true') \
    .config("spark.sql.warehouse.dir", "/user/itv016478/warehouse") \
    .enableHiveSupport() \
    .master('yarn') \
    .getOrCreate()


In [2]:
from pyspark.sql import SparkSession 
import getpass 
username = getpass.getuser() 
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/itv016478/warehouse"). \
config('spark.shuffle.useOldFetchProtocol','true'). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [3]:
spark

In [4]:
spark.sql("DESCRIBE FORMATTED loans_enq").show(truncate=False)


AnalysisException: Table or view not found for 'DESCRIBE TABLE': loans_enq; line 1 pos 0;
'DescribeRelation true, [col_name#0, data_type#1, comment#2]
+- 'UnresolvedTableOrView [loans_enq], DESCRIBE TABLE, true


In [5]:
!hadoop fs -ls /user/itv016478/lendingclubproject/cleaned/loans_def_detail_records_enq_parquet

Found 3 items
-rw-r--r--   3 itv016478 supergroup          0 2025-05-20 11:39 /user/itv016478/lendingclubproject/cleaned/loans_def_detail_records_enq_parquet/_SUCCESS
-rw-r--r--   3 itv016478 supergroup   77108077 2025-05-20 11:39 /user/itv016478/lendingclubproject/cleaned/loans_def_detail_records_enq_parquet/part-00000-21a4d4ec-5b31-4bff-b8fe-e7f23210b448-c000.snappy.parquet
-rw-r--r--   3 itv016478 supergroup   74124369 2025-05-20 11:39 /user/itv016478/lendingclubproject/cleaned/loans_def_detail_records_enq_parquet/part-00001-21a4d4ec-5b31-4bff-b8fe-e7f23210b448-c000.snappy.parquet


In [6]:
!hadoop fs -head /user/itv016478/lendingclubproject/cleaned/loans_def_detail_records_enq_parquet/part-00000-21a4d4ec-5b31-4bff-b8fe-e7f23210b448-c000.snappy.parquet

PAR1 �����|,�� 6 (@fffce59cff467895296e8ba124e293e31139c10eaaaf6eccb5d84f016fcd9a6d@0000f31fb4726ace47a52e7bbfa36a96551c7afcc899e9e61517806e7a4aca4e   ��@�   ��@   d86a1c7244ace602c0fa0b6fd301a2a835cde5436d32eb6c6007ce00ed8dc149@   25b8cd9ff47e50eea5be7b7966c943e0c72ee8cdc3d6761f5ef0abc280383180@   f9773942305b2dc9c4258b0bd3081d263ed8b9c8d955755f498aacd733b8390c@   b22f99c198f3738a7b6391fa5e4fefa5e81a85250c6a4a50eb2302c090873aec!�4b3b7068c15b0e0467c408030b1778347c9bcaed5d5f765e65779c68d5e8a14c@   0c7a9f8a6d5b209d2e724cf121a7bf3858bbf540815e72f4ac6b43d2a1fde7ff@   f1cc7e6ae9c73514875151657535e5f8d62b7669ceb8282fc4be0a204fb31366@   8f8faa7eeb2a1ee41e6f50ed19c931f55104321690319ac398c289ca25217ec2@   dad3f5ea8788186cb4763a5d9fb424c281698938aeb68c7be9bb337045c59886@   a20867c09851262024b8b5d1e4d5efc95256018c3f5859baa8256bc655e6a82e@   b62a92faa291b619c7b60c78a3a71e2eabd22bbc9613b77334a9aa20ab5dc27c@   9468bb9f4c7e39e7246c8ff2b80f3071ef43e8b50f542f14840e757b6dfa1795@   56a52

In [7]:
df = spark.read.format("parquet").load("/user/itv016478/lendingclubproject/cleaned/loans_def_detail_records_enq_parquet/part-00000-21a4d4ec-5b31-4bff-b8fe-e7f23210b448-c000.snappy.parquet")
df.show()

+--------------------+-------+--------------------+--------------+
|           member_id|pub_rec|pub_rec_bankruptcies|inq_last_6mths|
+--------------------+-------+--------------------+--------------+
|d86a1c7244ace602c...|    1.0|                 1.0|             0|
|25b8cd9ff47e50eea...|    0.0|                 0.0|             1|
|f9773942305b2dc9c...|    1.0|                 1.0|             0|
|b22f99c198f3738a7...|    0.0|                 0.0|             0|
|4b3b7068c15b0e046...|    0.0|                 0.0|             4|
|0c7a9f8a6d5b209d2...|    1.0|                 1.0|             0|
|f1cc7e6ae9c735148...|    0.0|                 0.0|             0|
|8f8faa7eeb2a1ee41...|    0.0|                 0.0|             0|
|dad3f5ea8788186cb...|    0.0|                 0.0|             2|
|a20867c0985126202...|    0.0|                 0.0|             2|
|b62a92faa291b619c...|    2.0|                 1.0|             2|
|9468bb9f4c7e39e72...|    0.0|                 0.0|           

In [8]:
# slow acccess with newer data ( max 1 day old)
spark.sql("""
CREATE OR REPLACE view itv016478_lending_clubb.customers_loan_v as SELECT
l.loan_id, c.member_id, c.emp_title ,c.emp_length, c.home_ownership ,c.annual_inc, c.addr_state, c.zip_code, c.country, c.grade, c.sub_grade, c.verification_status, c.tot_hi_cred_lim, c.application_type, c.annual_inc_joint, c.verification_status_joint,
l.loan_amount, l.funded_amount, l.loan_term_months, l.interest_rate,l.monthly_installment, l.issue_date, l.loan_status, l.loan_purpose, 
r.total_principal_received, r.total_interest_received, r.total_late_fee_received, r.total_payment_received, r.last_payment_amount, r.last_payment_date, r.next_payment_date,
d.delinq_2yrs ,d.delinq_amnt , d.mths_since_last_delinq ,
e.pub_rec, e.pub_rec_bankruptcies, e.inq_last_6mths

FROM itv016478_lending_clubb.customers c
LEFT JOIN itv016478_lending_clubb.loans l ON c.member_id = l.member_id
LEFT JOIN itv016478_lending_clubb.loans_repay r ON l.loan_id = r.loan_id
LEFT JOIN itv016478_lending_clubb.loans_delinq d ON c.member_id = d.member_id
LEFT JOIN itv016478_lending_clubb.loans_enqq e ON c.member_id = e.member_id
""")

""


In [9]:
spark.sql("select * from itv016478_lending_clubb.customers_loan_v")

loan_id,member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint,loan_amount,funded_amount,loan_term_months,interest_rate,monthly_installment,issue_date,loan_status,loan_purpose,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,delinq_2yrs,delinq_amnt,mths_since_last_delinq,pub_rec,pub_rec_bankruptcies,inq_last_6mths
71175798,000c8875b71a6b47c...,Director of Gvt O...,1,MORTGAGE,null,null,null,null,A,A4,Not Verified,null,Individual,null,null,3000.0,3000.0,null,7.39,93.17,Feb-2016,Fully Paid,debt_consolidation,3000.0,114.47,0.0,3114.47,2647.45,Aug-2016,null,1,0.0,13,0.0,0.0,0
139313300,000fc98fc1ca5faa3...,coord,10,MORTGAGE,null,null,null,null,A,A3,Not Verified,null,Joint App,null,null,20000.0,20000.0,null,7.21,619.47,Aug-2018,Current,credit_card,3558.76,769.52,0.0,4328.28,619.47,Mar-2019,Apr-2019,null,null,null,0.0,0.0,0
143195702,0012728d9f616bdf2...,null,6,OWN,null,null,null,null,A,A5,Not Verified,null,Individual,null,null,9000.0,9000.0,null,8.46,283.95,Dec-2018,Current,debt_consolidation,666.17,181.45,0.0,847.62,283.95,Mar-2019,Apr-2019,0,0.0,24,0.0,0.0,0
2284564,00151ece27c7ca280...,Northglenn Police...,8,RENT,null,null,null,null,C,C1,Not Verified,null,Individual,null,null,12800.0,12800.0,null,14.33,439.53,Nov-2012,Fully Paid,debt_consolidation,12800.0,2192.68,0.0,14992.676,7529.95,Jun-2014,null,0,0.0,29,0.0,0.0,1
3845340,002800d49886390d2...,RehabCare,2,RENT,null,null,null,null,D,D3,Not Verified,null,Individual,null,null,15000.0,15000.0,null,18.75,547.95,Mar-2013,Charged Off,debt_consolidation,12856.06,4702.54,0.0,17835.81,547.95,Dec-2015,null,null,null,null,0.0,0.0,0
360493,003715c2aec34dd43...,Apex Technologies...,1,MORTGAGE,null,null,null,null,C,C3,Not Verified,null,Individual,null,null,12000.0,12000.0,null,12.41,400.93,Oct-2008,Fully Paid,debt_consolidation,12000.0,208.02,0.0,12208.02,1.07,Apr-2009,null,null,null,null,0.0,0.0,2
95004059,003769d7f54c7859e...,Trainer,10,MORTGAGE,null,null,null,null,E,E1,Verified,null,Individual,null,null,10000.0,10000.0,null,22.74,280.42,Dec-2016,Charged Off,medical,90.92,176.87,0.0,1492.05,280.42,Jan-2017,null,1,0.0,20,0.0,0.0,0
142641777,0037bb910c0a758f5...,Network Engineer,2,RENT,null,null,null,null,C,C3,Source Verified,null,Individual,null,null,2000.0,2000.0,null,15.02,69.36,Oct-2018,Current,other,227.25,116.21,0.0,343.46,69.36,Mar-2019,Apr-2019,0,0.0,66,0.0,0.0,0
91518424,003d7bee408492f11...,Flight Attendant,10,RENT,null,null,null,null,A,A5,Not Verified,null,Individual,null,null,10000.0,10000.0,null,7.99,313.32,Oct-2016,Current,other,7864.12,1213.28,0.0,9077.4,313.32,Mar-2019,Apr-2019,null,null,null,0.0,0.0,0
116883875,003e1e6cbd2920bbb...,Operations Superv...,9,MORTGAGE,null,null,null,null,A,A5,Source Verified,null,Individual,null,null,10000.0,10000.0,null,7.97,313.23,Aug-2017,Current,debt_consolidation,4980.61,970.21,0.0,5950.82,313.23,Mar-2019,Apr-2019,1,0.0,23,1.0,1.0,0


In [10]:
spark.sql("use itv016478_lending_clubb")

""


In [11]:
spark.sql("SHOW TABLES").show(truncate=False)

+-----------------------+---------------------+-----------+
|database               |tableName            |isTemporary|
+-----------------------+---------------------+-----------+
|itv016478_lending_clubb|customers            |false      |
|itv016478_lending_clubb|customers_loan_a     |false      |
|itv016478_lending_clubb|customers_loan_l     |false      |
|itv016478_lending_clubb|customers_loan_s     |false      |
|itv016478_lending_clubb|customers_loan_t     |false      |
|itv016478_lending_clubb|customers_loan_v     |false      |
|itv016478_lending_clubb|customers_loan_vv    |false      |
|itv016478_lending_clubb|customerstest        |false      |
|itv016478_lending_clubb|loans                |false      |
|itv016478_lending_clubb|loans_delinq         |false      |
|itv016478_lending_clubb|loans_enq            |false      |
|itv016478_lending_clubb|loans_enqq           |false      |
|itv016478_lending_clubb|loans_enqqtest       |false      |
|itv016478_lending_clubb|loans_new_delin

In [12]:
spark.sql("select * from customers")

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint,ingest_date_timestamp
42c0d86fde84607a0...,Store Manager,3,RENT,null,null,null,null,B,B3,Verified,null,Individual,null,null,null
64181326110c859ed...,Vet tech,10,MORTGAGE,null,null,null,null,D,D4,Source Verified,null,Individual,null,null,null
d39b9dd4bbfcdf3b9...,AMSA,5,RENT,null,null,null,null,D,D4,Verified,null,Individual,null,null,null
fdf9ad3a88dc0970f...,Admin,7,MORTGAGE,null,null,null,null,C,C4,Verified,null,Individual,null,null,null
a80d21bff6db10e40...,Associate V.P.,1,MORTGAGE,null,null,null,null,A,A1,Not Verified,null,Individual,null,null,null
e475c71f8f6859eb5...,machinist,5,MORTGAGE,null,null,null,null,B,B2,Not Verified,null,Individual,null,null,null
14f52f3fdc9ab7cf1...,Security Guard,1,MORTGAGE,null,null,null,null,C,C5,Verified,null,Individual,null,null,null
219b462ac15250b23...,Systems Analyst,10,OWN,null,null,null,null,B,B4,Verified,null,Individual,null,null,null
dbd9f7a2f51bf1f52...,Supervisor,8,MORTGAGE,null,null,null,null,B,B3,Source Verified,null,Individual,null,null,null
0f5552e3a3960848b...,RN Senior Manager,2,MORTGAGE,null,null,null,null,B,B1,Not Verified,null,Individual,null,null,null


In [13]:
spark.sql("select * from itv016478_lending_clubb.loans_repay")

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date
76003861,6423.8,5607.85,0.0,12031.65,574.88,Jan-2018,null
76263914,2400.0,323.18,0.0,2723.1802,905.18,Dec-2017,null
75537401,12600.0,1392.76,0.0,13992.757,3437.74,Aug-2018,null
75038986,7805.03,7583.71,0.0,15388.74,440.72,Mar-2019,Apr-2019
76301424,2614.78,1080.09,0.0,3694.87,153.89,Apr-2018,null
75333198,8664.09,3431.32,68.8,12164.21,343.9,Mar-2019,Apr-2019
76391453,35000.0,10266.01,0.0,45266.008,18884.35,Feb-2019,null
76363364,15000.0,2093.11,0.0,17093.107,4648.59,Aug-2018,null
76272510,14463.21,11178.79,110.13,25752.13,734.18,Mar-2019,Apr-2019
76304116,4800.0,1051.65,0.0,5851.6523,1761.59,Dec-2017,null


In [14]:
spark.sql("select * from itv016478_lending_clubb.loans_delinq")

member_id,delinq_2yrs,delinq_amnt,mths_since_last_delinq
25b8cd9ff47e50eea...,0,0.0,51
b22f99c198f3738a7...,0,0.0,72
4b3b7068c15b0e046...,0,0.0,68
9468bb9f4c7e39e72...,1,0.0,7
a14a75a803373848c...,0,0.0,68
c1573ce869bafcbe9...,0,0.0,30
7a748acb9cb11d073...,0,0.0,40
3029a35037c3d47f2...,0,0.0,59
989b1622ddf441235...,0,0.0,75
69d87d14c315f222d...,0,0.0,55


In [15]:
spark.sql("select * from itv016478_lending_clubb.loans_enqq")

member_id,pub_rec,pub_rec_bankruptcies,inq_last_6mths
d86a1c7244ace602c...,1.0,1.0,0
25b8cd9ff47e50eea...,0.0,0.0,1
f9773942305b2dc9c...,1.0,1.0,0
b22f99c198f3738a7...,0.0,0.0,0
4b3b7068c15b0e046...,0.0,0.0,4
0c7a9f8a6d5b209d2...,1.0,1.0,0
f1cc7e6ae9c735148...,0.0,0.0,0
8f8faa7eeb2a1ee41...,0.0,0.0,0
dad3f5ea8788186cb...,0.0,0.0,2
a20867c0985126202...,0.0,0.0,2


In [16]:
spark.sql("select * from itv016478_lending_clubb.customers_loan_v").show()

+---------+--------------------+--------------------+----------+--------------+----------+----------+--------+-------+-----+---------+-------------------+---------------+----------------+----------------+-------------------------+-----------+-------------+----------------+-------------+-------------------+----------+-----------+------------------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+-----------+-----------+----------------------+-------+--------------------+--------------+
|  loan_id|           member_id|           emp_title|emp_length|home_ownership|annual_inc|addr_state|zip_code|country|grade|sub_grade|verification_status|tot_hi_cred_lim|application_type|annual_inc_joint|verification_status_joint|loan_amount|funded_amount|loan_term_months|interest_rate|monthly_installment|issue_date|loan_status|      loan_purpose|total_principal_received|total_interest_received|total_lat

In [17]:
#quick access with little older data in table form (max 7 days old)
spark.sql("""CREATE TABLE itv016478_lending_clubb.customers_loan_a
USING PARQUET
AS SELECT
    l.loan_id,
    c.member_id,
    c.emp_title,
    c.emp_length,
    c.home_ownership,
    c.annual_inc,
    c.addr_state,
    c.zip_code,
    c.country,
    c.grade,
    c.sub_grade,
    c.verification_status,
    c.tot_hi_cred_lim,
    c.application_type,
    c.annual_inc_joint,
    c.verification_status_joint,
    l.loan_amount,
    l.funded_amount,
    l.loan_term_months,
    l.interest_rate,
    l.monthly_installment,
    l.issue_date,
    l.loan_status,
    l.loan_purpose,
    r.total_principal_received,
    r.total_interest_received,
    r.total_late_fee_received,
    r.total_payment_received,
    r.last_payment_amount,
    r.last_payment_date,
    r.next_payment_date,
    d.delinq_2yrs,
    d.delinq_amnt,
    d.mths_since_last_delinq,
    e.pub_rec,
    e.pub_rec_bankruptcies,
    e.inq_last_6mths
FROM itv016478_lending_clubb.customers c
LEFT JOIN itv016478_lending_clubb.loans l ON c.member_id = l.member_id
LEFT JOIN itv016478_lending_clubb.loans_repay r ON l.loan_id = r.loan_id
LEFT JOIN itv016478_lending_clubb.loans_delinq d ON c.member_id = d.member_id
LEFT JOIN itv016478_lending_clubb.loans_enqq e ON c.member_id = e.member_id
""")


AnalysisException: Table itv016478_lending_clubb.customers_loan_a already exists. You need to drop it first.

In [18]:
#spark.sql("""
#CREATE Table itv016478_lending_clubb.customers_loan_s as SELECT
#l.loan_id, c.member_id, c.emp_title ,c.emp_length, c.home_ownership ,c.annual_inc, c.addr_state, c.zip_code, c.country, c.grade, c.sub_grade, c.verification_status, c.tot_hi_cred_lim, c.application_type, c.annual_inc_joint, c.verification_status_joint,
#l.loan_amount, l.funded_amount, l.loan_term_months, l.interest_rate,l.monthly_installment, l.issue_date, l.loan_status, l.loan_purpose, 
#r.total_principal_received, r.total_interest_received, r.total_late_fee_received, r.total_payment_received, r.last_payment_amount, r.last_payment_date, r.next_payment_date,
#d.delinq_2yrs ,d.delinq_amnt , d.mths_since_last_delinq ,
#e.pub_rec, e.pub_rec_bankruptcies, e.inq_last_6mths

#FROM itv016478_lending_clubb.customers c
#LEFT JOIN itv016478_lending_clubb.loans l ON c.member_id = l.member_id
#LEFT JOIN itv016478_lending_clubb.loans_repay r ON l.loan_id = r.loan_id
#LEFT JOIN itv016478_lending_clubb.loans_delinq d ON c.member_id = d.member_id
#LEFT JOIN itv016478_lending_clubb.loans_enq e ON c.member_id = e.member_id
#""")

In [19]:
spark.sql("select * from  itv016478_lending_clubb.customers_loan_a ")

loan_id,member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint,loan_amount,funded_amount,loan_term_months,interest_rate,monthly_installment,issue_date,loan_status,loan_purpose,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,delinq_2yrs,delinq_amnt,mths_since_last_delinq,pub_rec,pub_rec_bankruptcies,inq_last_6mths
93802960,0009423060a3ddbfc...,Supervisor,10,MORTGAGE,null,null,null,null,D,D3,Verified,null,Individual,null,null,null,null,null,null,null,null,Fully Paid,null,20525.0,4381.46,0.0,24906.463,13644.06,Apr-2018,null,0,0.0,30,0.0,0.0,2
58752276,000b587ab90fc77bf...,Medical Transcrip...,8,MORTGAGE,null,null,null,null,F,F5,Verified,null,Individual,null,null,null,null,null,null,null,null,Charged Off,null,2575.56,5143.68,0.0,9306.77,455.36,Feb-2017,null,0,0.0,66,3.0,1.0,2
60275689,00109c0225db1912f...,Assistant Superin...,10,OWN,null,null,null,null,B,B1,Source Verified,null,Individual,null,null,null,null,null,null,null,null,Current,null,9842.52,2972.33,0.0,12814.85,305.44,Mar-2019,Apr-2019,9,0.0,15,0.0,0.0,1
103861199,0011d8e5fab8e99ce...,CBP Officer,5,RENT,null,null,null,null,C,C4,Verified,null,Individual,null,null,null,null,null,null,null,null,Fully Paid,null,3000.0,435.82,0.0,3435.8242,2192.94,May-2018,null,null,null,null,0.0,0.0,0
104954334,0013a384974fdd540...,Sales,1,OWN,null,null,null,null,A,A3,Not Verified,null,Individual,null,null,null,null,null,null,null,null,Current,null,7855.3,1262.27,0.0,9117.57,396.64,Mar-2019,Apr-2019,4,0.0,5,0.0,0.0,0
88976275,0014532e28e53c904...,front desk Agent,6,RENT,null,null,null,null,C,C4,Source Verified,null,Individual,null,null,null,null,null,null,null,null,Current,null,4010.13,1144.52,0.0,5154.65,172.09,Mar-2019,Apr-2019,null,null,null,0.0,0.0,0
135752464,0017d3c6ad0327c9e...,Staffing Coordinator,10,OWN,null,null,null,null,C,C5,Source Verified,null,Individual,null,null,null,null,null,null,null,null,Current,null,2475.63,2808.1,0.0,5283.73,589.52,Mar-2019,Apr-2019,0,0.0,73,0.0,0.0,2
65825167,001c3b346d130a15a...,Chief Operating O...,5,RENT,null,null,null,null,A,A1,Verified,null,Individual,null,null,null,null,null,null,null,null,Fully Paid,null,18000.0,1501.07,0.0,19501.066,541.92,Nov-2018,null,null,null,null,0.0,0.0,1
70877445,00292405582f71c64...,Due Diligence Ana...,1,OWN,null,null,null,null,C,C4,Source Verified,null,Individual,null,null,null,null,null,null,null,null,Fully Paid,null,10000.0,2340.82,0.0,12340.815,360.37,Feb-2019,null,null,null,null,1.0,0.0,1
137374764,002c8ff9357ee9eb9...,Machine operator,1,RENT,null,null,null,null,B,B5,Verified,null,Individual,null,null,null,null,null,null,null,null,Current,null,3848.8,1641.73,0.0,5490.53,778.69,Mar-2019,Apr-2019,1,0.0,22,0.0,0.0,0


In [20]:
spark.stop()